In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geojson
import pickle

from geojson import Feature, Point, FeatureCollection

## Assets

In [3]:
geojson_localidad_file = '/Users/anamaria/Desktop/dev/security_project/assets/localidades_polygon.json'

with open(geojson_localidad_file) as f:
    geojson_localidad = geojson.load(f)

## Built-in

In [43]:
def generate_centroids(geojson_object):
    centroids = {}    
    for i in range(0,len(geojson_object['features'])):
        list_coords =list(map(lambda x: tuple(x), geojson_object['features'][i]['geometry']['coordinates'][0][0]))
        key = geojson_object['features'][i]['properties']['LocNombre']
        centroids[key] = Polygon(list_coords).centroid
    return centroids

In [57]:
def get_localidad_centroid(centroids_list, localidad_name):
    return centroids_list[localidad_name]

In [68]:
def generate_localidad_id(geojson_object):
    localidades_ids = {}
    for i in range(0,len(geojson_object['features'])):
        key = geojson_object['features'][i]['properties']['LocNombre']
        localidades_ids[key] = geojson_object['features'][i]['properties']['LocCodigo']
    return localidades_ids

In [71]:
def get_localidad_id(localidades_ids, localidad_name):
    return localidades_ids[localidad_name]

## Data

In [88]:
## Results of predicatbility experiment_08 (using NUSE sample dataset; 2017-2018)

In [31]:
infile = open('/Users/anamaria/Desktop/dev/security_project/periodicity_experiments/visualization/predictability_localidades.pkl','rb')
predictability_localidades = pickle.load(infile)
infile.close()

In [87]:
predictability_localidades = predictability_localidades[predictability_localidades['crime_level']=='3']

## Generate geojson

In [69]:
localidades_centroids = generate_centroids(geojson_localidad)
localidades_ids = generate_localidad_id(geojson_localidad)

In [85]:
features_list = list()

localidades_list = predictability_localidades.localidad.unique()
for localidad in localidades_list:
    localidad_centroid = get_localidad_centroid(localidades_centroids, localidad)
    localidad_id = get_localidad_id(localidades_ids, localidad)
    localidad_feature = Feature(geometry = localidad_centroid,
                                id = int(localidad_id),
                                properties = {'localidad_name': localidad,
                                              'localidad_code': int(localidad_id),
                                              'object_id': int(localidad_id),
                                              'predictability': float(predictability_localidades.loc[predictability_localidades['localidad']==localidad].predictability.values[0]),
                                              'contingency': float(predictability_localidades.loc[predictability_localidades['localidad']==localidad].contingency.values[0]),
                                              'constancy': float(predictability_localidades.loc[predictability_localidades['localidad']==localidad].constancy.values[0])
                                             }
                               )
    features_list.append(localidad_feature)
    

In [90]:
predictability_localidades_geojson = FeatureCollection(features_list)

In [91]:
geojson_file = open("./choropleth_predictability_localidades.geojson", "a")
geojson_file.write(str(predictability_localidades_geojson))
geojson_file.close()

In [6]:


def generate_geojson_centers_incidents_upz(df):
    
    lst_features = list()
    
    for i in range(len(UPZ_ids)):
        num_incidents = 0
        exist_incidents = len(df[df['UPZ'] == UPZ_names[i]]['NUM'].values)
        if (exist_incidents > 0):
            num_incidents = int(df[df['UPZ'] == UPZ_names[i]]['NUM'].values)          
        center_upz = Point((clons[i], clats[i]))
        feature_upz = Feature(geometry = center_upz, 
                              id = UPZ_ids[i], properties = {'upz_name': UPZ_names[i],
                                                             'incidents': num_incidents,
                                                             'COD_UPZ': UPZ_cods[i],
                                                             'OBJECTID': UPZ_objids[i]
                                                            }) 
        lst_features.append(feature_upz)
        
    return FeatureCollection(lst_features)
    
centers_incidents_upz = generate_geojson_centers_incidents_upz(df_centroids)

In [7]:
f = open("./heatmap.geojson", "a")
f.write(str(centers_incidents_upz))
f.close()